In [1]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

In [2]:
prices_df = pd.read_csv('data/prices_formatted.csv')
prices_df['description'] = prices_df.apply(lambda df: f'{df.local_time} - Price: {df.mid:,.2f}', axis=1)

In [3]:
tweets_df = pd.read_csv('data/tweets_formatted.csv')
tweets_df['tweet'] = tweets_df.apply(lambda df: f'{df.num_coins:,} coins (${df.usd_value:,}) {df.description}', axis=1)

def _color(usd_value):
    if usd_value >= 100000000:
        return 'lightgreen'
    elif usd_value >= 50000000:
        return 'lightblue'
    else:
        return 'yellow'
tweets_df['color'] = tweets_df['usd_value'].apply(_color)

In [4]:
def get_price_fig(prices_df):
    return px.line(prices_df, 
                    x='unix_time', 
                    y='mid',
                    hover_name='description',
                    color_discrete_sequence=['rgb(57,118,175)'])

def get_tweet_fig(tweets_df, price_min, price_max):
    # Add markers with hover text at the top and bottom of the figure so you can read the tweet
    def _get_tweet_markers(y_pos):
        return go.Figure(go.Scatter(x=tweets_df['unix_time'],
                                    y=[y_pos for _ in range(tweets_df.shape[0])],
                                    mode='markers',
                                    marker_color=tweets_df['color'],
                                    text=tweets_df['tweet']))
    tweet_fig = go.Figure(_get_tweet_markers(price_min).data + _get_tweet_markers(price_max).data)
    
    return tweet_fig
    
def add_tweet_lines(tweets_df, fig):
    # Add vertical lines for each tweet
    for (_, (tweet, unix_time, color)) in tweets_df[['tweet', 'unix_time', 'color']].iterrows():
        fig.add_vline(x=unix_time, line_color=color)
    
    return fig

def add_legend(fig, xmin, xmax, ymin, ymax):

    spacing = (ymax - ymin) / 16
    x = [xmax, xmax, xmax]
    y = [ymin + (spacing * 2), ymin + spacing, ymin]
    
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode="markers+text",
        name="Markers and Text",
        text=[" >$100M", " >$50M", " <$50M"],
        marker_color=['lightgreen', 'lightblue', 'yellow'],
        textposition="middle right"
    ))
    return fig

In [7]:
def get_all_plots(prices_df, tweets_df, asset):
    
    plots = []
    
    prices_df = prices_df[(prices_df.asset == asset)]
    tweets_df = tweets_df[(tweets_df.asset == asset)]
    
    # Slice the price window into smaller plots
    slices = 7
    window_size = (prices_df.shape[0] // slices) + 1
    
    for i in range(slices):
        start = i * window_size
        end = (i + 1) * window_size 
        
        price_window_df = prices_df[start:end]
        
        start_time, end_time = (price_window_df['unix_time'].min(), price_window_df['unix_time'].max())
        filter_tweet_cond = (
            (tweets_df['unix_time'] >= start_time)
            & (tweets_df['unix_time'] <= end_time)
        )
        tweet_window_df = tweets_df[filter_tweet_cond]
    
        price_fig = get_price_fig(price_window_df)

        price_min = price_window_df['mid'].min()
        price_max = price_window_df['mid'].max()

        tweet_fig = get_tweet_fig(tweet_window_df, price_min, price_max)

        fig = go.Figure(tweet_fig.data + price_fig.data)
        fig = add_tweet_lines(tweet_window_df, fig)

        start_time_string = datetime.fromtimestamp(start_time).strftime("%Y-%m-%d %I:%M:%S %p")
        end_time_string = datetime.fromtimestamp(end_time).strftime("%Y-%m-%d %I:%M:%S %p")

        fig.update_layout(
            title=f'{asset} - Prices and Whale Alerts by Time - {start_time_string} to {end_time_string}',
            xaxis_title="Time",
            yaxis_title="Price",
            showlegend=False
        )
        
        fig = add_legend(fig, start_time, end_time, price_min, price_max)
        
        plots.append(fig)
        
    return plots

In [8]:
for asset in ['BTC', 'ETH', 'USDT']:
    
    if 'plots' not in os.listdir('.'):
        os.mkdir('plots')
    
    file_path = f'plots/{asset}.html'
    
    with open(file_path, 'w') as f:
        f.write(f'<h1>{asset}<h1>')
        
        for plot in get_all_plots(prices_df, tweets_df, 'BTC'):
            f.write(plot.to_html())
    